# Visualization Heavy rainfall (code)

## Install libraries

In [2]:
# Python libraries
import datetime
import os
import glob
import requests
import zipfile

# Read dataset
import xarray as xr

# Projections
import cartopy.crs as ccrs
import pyproj

# Plots
import plotly.express as px
import plotly.graph_objs as go
import matplotlib as mpl
from matplotlib.colors import from_levels_and_colors
%matplotlib inline

# Leafmap
from leafmap import Map
from leafmap.toolbar import tool_template
from leafmap.colormaps import plot_colormap
from ipyleaflet import DrawControl, WidgetControl, LayerGroup, Marker
from localtileserver import get_leaflet_tile_layer

# Jupyter widgets
from ipywidgets import Layout, interactive_output, interactive, HBox, VBox, Dropdown
import ipywidgets as widgets


In [4]:
def download_data():
    # Download the zip file
    url_data = "http://gebrada.upc.es/climaax/heavy_rainfall_data.zip"
    tmp_file = "heavy_rainfall_data.zip"
    response = requests.get(url_data)
    open(tmp_file, "wb").write(response.content)
    
    # Uncompress data 
    with zipfile.ZipFile(tmp_file, 'r') as zip_ref:
        zip_ref.extractall(DATADIR)

    # Move the data to DATADIR and delete zip file
    os.remove(tmp_file)


In [5]:
# Palettes definition for MAP
def define_palettes():
    # Colors for the absolute values
    pal_abs_rgb = ['#ffffff00', '#d6e2ffff', '#8db2ffff', '#626ff7ff', '#0062ffff', '#019696ff', '#01c634ff',
                   '#63ff01ff', '#c6ff34ff', '#ffff02ff', '#ffc601ff', '#ffa001ff', '#ff7c00ff', '#ff1901ff', '#a20a28ff',
                   '#9b159dff', '#d294d3ff','#f6e9f6ff']
    # Thresholds for the absolute values (durations 3 and 6 hours)
    pal_abs_thresh_3_6 = [0, 10, 20., 25., 30., 35., 40., 45., 50., 60., 70., 80., 90., 100., 110., 120., 130., 140.]
    # Thresholds for the absolute values (durations 12 and 24 hours)
    pal_abs_thresh_12_24 = [0., 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 500]
    # Thresholds for the relative value
    pal_rel_thresh = [-15, -10, -5, -2, -1, 0, 1, 2, 5, 10, 15]

    # CMAPs and NORMs for all the palettes
    cmap_abs_3_6, norm_abs_3_6 = from_levels_and_colors(pal_abs_thresh_3_6, pal_abs_rgb, extend='max')
    cmap_abs_12_24, norm_abs_12_24 = from_levels_and_colors(pal_abs_thresh_12_24, pal_abs_rgb, extend='max')
    cmap_rel = mpl.cm.RdYlGn
    norm_rel = mpl.colors.BoundaryNorm(pal_rel_thresh, cmap_rel.N, extend='both')

    # Mappables for each palette (used to plot the colorbar in the map)
    mappable_abs_3_6 = mpl.cm.ScalarMappable(norm = norm_abs_3_6, cmap = cmap_abs_3_6)
    mappable_abs_value_24 = mpl.cm.ScalarMappable(norm=norm_abs_12_24, cmap=cmap_abs_12_24)
    mappable_rel = mpl.cm.ScalarMappable(norm=norm_rel, cmap=cmap_rel)
    
    # register as a matplotlib cmap so it can be called later
    mpl.colormaps.register(cmap_abs_3_6, name="cmap_abs_3_6")
    mpl.colormaps.register(cmap_abs_12_24, name="cmap_abs_12_24")
    mpl.colormaps.register(cmap_rel, name="cmap_rel")
    
    return {
        'cmap_abs_3_6' : mappable_abs_3_6,
        'cmap_abs_12_24' : mappable_abs_value_24,
        'cmap_rel' : mappable_rel
    }


In [6]:
def get_static_attributes():
    durations = [
        ('3h', 3),
        ('6h', 6),
        ('12h', 12),
        ('24h', 24)
    ]
    
    frequencies = [
        ('2 years', 2),
        ('5 years', 5),
        ('10 years', 10),
        ('25 years', 25),
        ('50 years', 50),
        ('100 years', 100),
        ('200 years', 200),
        ('500 years', 500)
    ]
    
    model_years = { 
        'ICHEC-EC-EARTH': [
            ('Baseline', datetime.datetime(2005, 1, 1)),
            ('2010-2040', datetime.datetime(2040, 1, 1)),
            ('2040-2070', datetime.datetime(2070, 1, 1)),
            ('2070-2100', datetime.datetime(2100, 1, 1)),
        ],
        'MOHC-HADGEM': [
            ('Baseline', datetime.datetime(2005, 1, 1)),
            ('2010-2040', datetime.datetime(2040, 1, 1)),
            ('2040-2070', datetime.datetime(2070, 1, 1)),
            ('2070-2100', datetime.datetime(2099, 1, 1)),
        ]
    }
    
    model_gcms = {
        'ICHEC-EC-EARTH': [
            ('RCP 8.5','rcp85')
        ], 
        'MOHC-HADGEM': [
            ('RCP 8.5','rcp85')
        ]
    }
    return durations, frequencies, model_years, model_gcms


In [7]:
def load_layers():
    layers = {}
    # Get all the models and the years to show as images
    for model, years in MODEL_YEARS.items():
        layers[model] = {} 
        # Loop for all the years
        for year_label, year_dt in years:
            layers[model][year_label] = {}
            # Get all the gcms available for the model
            for _, gcm in MODEL_GCMS[model]:
                gcm_str = f"{'historical' if year_dt.year == 2005 else gcm}"  # The year 2005 is stored in the historical folder (same for all the gcms)
                # Loop for all durations
                for dur_str, dur in DURATIONS:
                    layers[model][year_label][dur] = {}
                    # Loop for all the frequencies
                    for _, freq in FREQUENCIES:
                        # Build the path to the tiff file
                        path_tiff = os.path.join(DATADIR, f"cordex_{model}", gcm_str, "tiff")
                        # Load the absolute value tiff
                        layers[model][year_label][dur][freq] = {
                            'abs_value': get_leaflet_tile_layer(
                                glob.glob(f"{path_tiff}/idf_{dur_str}_{model}_{gcm_str}_*_*{year_dt:%Y}*_*{freq}y.tiff")[0],
                                port = 9999,
                                band = 1,
                                cmap = 'cmap_abs_3_6' if dur in [3, 6] else 'cmap_abs_12_24',
                                opacity = 0.6
                            )
                        }
                        # Load the relative change tiff
                        if gcm_str != 'historical':
                            layers[model][year_label][dur][freq]['rel_change'] = get_leaflet_tile_layer(
                                glob.glob(f"{path_tiff}/relative_change/idf_{dur_str}_{model}_{gcm_str}_*_*{year_dt:%Y}*_*{freq}y_dif.tiff")[0],
                                port = 9999,
                                band = 1,
                                cmap = 'cmap_rel',
                                opacity = 0.6
                            )
    return layers


In [8]:
# Multiple GCMs and RCPs
def load_model(gcm):
    d_ts = {}
    d_idf = {}
    for _, rcp in MODEL_GCMS[gcm]:
        # Read Maximum accumulations
        f_max = glob.glob(os.path.join(DATADIR, f'cordex_{gcm}', rcp, 'pr_annualMax*.nc'))
        da = xr.open_mfdataset(f_max, decode_coords='all')
        d_ts[rcp] = da.pr_max.rolling(time = 30).mean().dropna(dim = 'time', how = 'all')

        # Read IDFs
        f_idf_hist = glob.glob(os.path.join(DATADIR, f'cordex_{gcm}', 'historical', 'idf*.nc'))
        f_idf_proj = glob.glob(os.path.join(DATADIR, f'cordex_{gcm}', rcp, 'idf*.nc'))
        f_idf = f_idf_hist + f_idf_proj
        da = xr.open_mfdataset(f_idf, decode_coords='all')
        d_idf[rcp] = da.idf
    return d_ts, d_idf


In [9]:
# Create map object
def create_ui():
    #######################################################
    #####   Functions that handle the interactivity   #####
    #######################################################

    # Function that checks if it is posible to show the relative change (only available in the projections)
    def handle_abs_value_buttons(change):
        nonlocal abs_value_widget
        # If it is a projection (not Baseline) enable the relative change option
        if change.new != 'Baseline':
            abs_value_widget.disabled = False       
        # If it is the Baseline disable the relative change option and force the map to absolute
        else:
            abs_value_widget.disabled = True
            abs_value_widget.value = 'abs_value'
    
    # Function that handles the click (add pointer and plot charts)
    marker_layer = None
    coords = None
    def handle_click(**kwargs):
        nonlocal marker_layer, coords
        if kwargs.get('type') == 'click':
            # Get coordinates
            coords = kwargs.get('coordinates')
            # Remove previous map marker (if exists)
            m.remove_layer(marker_layer)
            # Add new map marker
            marker_layer = Marker(location=coords)
            m.add(marker_layer)
            # Call the function to plot the carts
            show_plots()
    
    # Function that manage which layer is visible on the map
    current_layer = None
    def update_layers(value):
        nonlocal current_layer
        # Remove current layer (if it exists)
        if current_layer is not None:
            m.remove_layer(current_layer)
        
        l = LAYERS[gcm_widget.value][years_widget.value][dur_widget.value][rp_widget.value][abs_value_widget.value]
        m.add(l)
        current_layer = l

    # Function to show the plots
    def show_plots():
        nonlocal rcp_widget, dur_widget, rp_widget, output_plots, coords
        
        # No coords selected (no plots to show)
        if coords is None:
            return

        # Transform lat and lon to rotated pole (dataset projection)
        crs = ccrs.RotatedPole(pole_latitude=39.25, pole_longitude=-162)
        transformer = pyproj.Transformer.from_crs('epsg:4326',crs)
        rlon, rlat = transformer.transform(*coords)
        
        with output_plots:
            # Delete charts already plotted
            output_plots.clear_output()
            
            # FIRST CHART
            # Get mean maximun precipitation for the selected duration, rcp, lat and lon
            da = DS_MAX[rcp_widget.value].sel(duration=dur_widget.value).sel(rlat = rlat, rlon = rlon, method = 'nearest')
            # Transform to dataset and rename the column to 'precipitation'
            df = da.to_dataframe().reset_index().rename(columns={'pr_max': 'precipitation'})
            # Add the colomn 'data source' that defines if the time is historic o projection
            df['data source'] = df['time'].apply(lambda x: 'projections' if x.year > 2005 else 'historic')
            # Plot a bar chart. X axis = time; Y axis = precipitation; Color = data source
            fig = px.bar(
                df,
                x = "time",
                y = "precipitation",
                color = 'data source',
                color_discrete_sequence = ['lightblue', 'lightgreen'],
                title = f'30y mean of maximum annual precipitation for {da.duration.values}h'
            )
            # Add Y axis title and legend
            fig.update_layout(
                yaxis = dict(title="precipitation (mm)", range=[10, max(df.precipitation)+5]),
                legend = dict(orientation="h", yanchor="bottom",  y=1.02, xanchor="right",x=1)
            )
            # Show figure
            fig.show()

            # SECOND CHART
            # Get return period for the selected duration, rcp, lat and lon
            da2 = DS_IDF[rcp_widget.value].sel(duration=dur_widget.value, frequency=rp_widget.value).sel(rlat = rlat, rlon = rlon, method = 'nearest')
            # Get the RP value and the confidence intervals as dataframes
            df_upper = da2.sel(ci="ci_upper").to_dataframe().reset_index()
            df_lower = da2.sel(ci="ci_lower").to_dataframe().reset_index()
            df_value = da2.sel(ci="value").to_dataframe().reset_index()
            # Line for the RP
            center_trace = go.Scatter(
                x = df_value.time,
                y = df_value.idf,
                mode = 'lines',
                name = 'Value',
                line = dict(color='blue'),
                showlegend = False
            )
            # Line for the upper CI
            upper_trace = go.Scatter(
                x = df_upper.time,
                y = df_upper.idf,
                mode = 'lines',
                name = 'Upper CI',
                line = dict(width=0),
                showlegend = False
            )
            # Line for the lower CI
            lower_trace = go.Scatter(
                x = df_lower.time,
                y = df_lower.idf,
                mode = 'lines',
                name = 'Lower CI',
                line = dict(width=0),
                fillcolor = 'rgba(68, 68, 68, 0.3)',
                fill = 'tonexty',
                showlegend = False
            )
            # Create the figure with the 3 lines
            fig2 = go.Figure(data=[center_trace, upper_trace, lower_trace])
            # Add titles
            fig2.update_layout(
                title = f'Rainfall intensity data for {da2.duration.values}h duration for a {da2.frequency.values}y return period',
                xaxis = dict(title='time'),
                yaxis=dict(title='intensity (mm)', range=[10, max(df_upper.idf)+5])
            )
            # Show figure
            fig2.show()
    
            # TABLE
            # Get years to show in the table
            years_table = [a[1].year for a in MODEL_YEARS[MODEL]]
            # Get return period for the selected duration, rcp, lat and lon
            da3 = DS_IDF[rcp_widget.value].sel(duration=dur_widget.value, time=DS_IDF[rcp_widget.value].time.dt.year.isin(years_table)).sel(rlat = rlat, rlon = rlon, method = 'nearest')
            # Get the data as a dataframe
            df_rp = da3.sel(ci="value").to_dataframe().reset_index().pivot(index='frequency', columns='time', values='idf')
            # Rename columns (just the year of the datetime)
            df_rp.columns = [item.year for item in df_rp.columns]
            # Put the frequency as a column
            df_rp = df_rp.reset_index().rename(columns={'frequency': 'Return Period (years)'})
            # Show only 2 decimals
            df_rp = df_rp.map(lambda x: round(x, 2))
            # Create the table
            tab = go.Table(
                header = dict(
                    values = list(df_rp.columns),
                    fill_color = 'lightblue',
                    align = 'left'
                ),
                cells = dict(
                    values = df_rp.transpose().values.tolist(),
                    fill_color = ['lightblue', 'white','white','white'],
                    align='left')
            )
            # Create a figure with the table
            fig3 = go.Figure(data=[tab]).update_layout(height=400, title = f'Intensities for {da2.duration.values}h duration')
            # Show the figure
            fig3.show()

    # Function to update both, layers and plots at the same time
    def update_layers_and_plots(value):
        update_layers(value)
        show_plots()

    # Function that loads the new model and updates the layers and plots
    def update_gcm(value):
        # Remove visible layer of the old model
        nonlocal current_layer
        m.remove_layer(current_layer)
        current_layer = None
        
        with output_plots:
            # Delete charts of the old model
            output_plots.clear_output()
        # Load the new model and update the layers and plots
        global MODEL
        MODEL = value.new
        load_model(MODEL)
        update_layers_and_plots(None)
        
    #######################################################
    #####               Create the map                #####
    #######################################################
    m = Map(
        center = (42.5531, 3),
        zoom = 6,
        layout = Layout(width='90%', height='800px', margin='0 0 0 5%'),
        draw_control = False,
        measure_control = False,
        attribution_control = False,
        layers_control = False,
        scale_control = False,
        toolbar_control = False
    )
    
    #######################################################
    ##### Create the widgets that goes inside the map #####
    #######################################################
    # Year (period) widget
    years_options = [('Baseline', 'Baseline'), ('2010-2040', '2010-2040'), ('2040-2070', '2040-2070'), ('2070-2100', '2070-2100')]
    years_widget = widgets.RadioButtons(
        options = years_options,
        value = years_options[0][1],
        disabled = False,
        layout = Layout(display='flex', width='auto', padding="10px 10px 10px 10px"),
    )
    years_label = widgets.Label(
        value = 'Period',
        style = {
            'font_size' : '15px',
            'font_style': 'bold',
        },
        layout = Layout(padding='0 5px 10px 5px')
    )
    
    # Selection of type of map (absolute values or relative change)
    abs_value_widget = widgets.RadioButtons(
        options = [('Absolute value', 'abs_value'), ('Relative change', 'rel_change')],
        value = 'abs_value',
        disabled = True,
        layout = Layout(display='flex', width='auto', padding="10px 10px 10px 10px"),
    )
    # Add the controls inside the map (topright)
    years_control = WidgetControl(widget=widgets.VBox([years_label, years_widget, abs_value_widget], layout=Layout(width='150px')), position='topright')
    m.add_control(years_control)


    #######################################################
    ####  Create the widgets that goes outside the map ####
    #######################################################
    # Define widgets to select the options (each widget has a label widget associated)
    # Layout for all dropdowns
    layout_select = widgets.Layout(
        display='flex',
        margin='5px 0px 5px 0px',
        max_width='90%'
                                    
    )
    
    # Duration Selector
    dur_widget = widgets.Dropdown(
        options= DURATIONS,
        value = DURATIONS[0][1],
        disabled=False,
        layout=layout_select
    )
    dur_label = widgets.HTML(value='<b>Duration</b>')
    dur_select = widgets.VBox([dur_label, dur_widget])
    
    # Return period (frequency) selector
    rp_widget = widgets.Dropdown(
        options=FREQUENCIES,
        value = FREQUENCIES[0][1],
        rows=8,   
        disabled=False,
        layout=layout_select
    )
    rp_label = widgets.HTML(value='<b>Return period</b>')
    rp_select = widgets.VBox([rp_label, rp_widget])
    
    # Global climate model selector
    gcm_widget = widgets.Dropdown(
        options = MODEL_GCMS.keys(),
        disabled = False,
        layout=layout_select
    )
    gcm_label = widgets.HTML(value='<b>Global Climate Model</b>')
    gcm_select = widgets.VBox([gcm_label, gcm_widget])
    
    # RCP selector
    rcp_widget = widgets.Dropdown(
        options=MODEL_GCMS['ICHEC-EC-EARTH'],
        value=MODEL_GCMS['ICHEC-EC-EARTH'][0][1],
        layout=layout_select
    )
    rcp_label = widgets.HTML(value='<b>Scenario</b>')
    rcp_select = widgets.VBox([rcp_label, rcp_widget])
        
    # Boxes for options and map
    options = widgets.HBox(
        [gcm_select, rcp_select, dur_select, rp_select],
        layout = Layout(justify_content='space-around', align_items='center', margin='10px 10px 10px 10px')
    )
    output_plots = widgets.Output(layout={'flex': '1'})
    widgets.VBox([options, m, output_plots])
    
    #######################################################
    #####  Link widgets with their function handlers  #####
    #######################################################
    # Map click (update charts)
    m.on_interaction(handle_click)
    
    # Widgets inside the map:
    # Year widget: 2 handlers (enable/disable options in widget and update layers)
    years_widget.observe(handle_abs_value_buttons, 'value')
    years_widget.observe(update_layers, 'value')
    # Absolute / relative widget: update layers
    abs_value_widget.observe(update_layers, 'value')

    # Widgets outside the map:
    rcp_widget.observe(update_layers_and_plots, 'value')
    dur_widget.observe(update_layers_and_plots, 'value')
    rp_widget.observe(update_layers_and_plots, 'value')
    gcm_widget.observe(update_gcm, 'value')
    
    # Show default layers 
    update_layers(None)
    return [options, m, output_plots]


In [10]:
print('Downloading data...')
BASEDIR = 'flashflood_workflow'
DATADIR = os.path.join(BASEDIR, 'data')
os.makedirs(DATADIR, exist_ok=True)
#download_data()

print('Creating palettes...')
MAPPABLES = define_palettes()

print('Getting static attributes...')
DURATIONS, FREQUENCIES, MODEL_YEARS, MODEL_GCMS = get_static_attributes()

print('Loading images...')
LAYERS = load_layers()

print('loading model data...')
MODEL = 'ICHEC-EC-EARTH'
DS_MAX, DS_IDF = load_model(MODEL)

print('creating user interface...')
WIDGETS_LIST = create_ui()
widgets.VBox(WIDGETS_LIST)

downloading data...
creating palettes...
getting static attributes...
loading images...


Address already in use
Port 9999 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'